In [2]:
import pandas as pd
from zipline.pipeline import Pipeline
from zipline.pipeline.data import USEquityPricing
from sharadar.pipeline.factors import (
    MarketCap,
    EV,
    Fundamentals
)
from sharadar.pipeline.engine import symbol, symbols, make_pipeline_engine
from zipline.pipeline.filters import StaticAssets
import time
import datetime

In [6]:
symbol('^VIX')

Equity(111630 [^VIX])

In [7]:
symbol('SPY')

Equity(118691 [SPY])

In [3]:
spe = make_pipeline_engine()

pipe_start = pd.to_datetime('2020-02-03', utc=True)
pipe_end = pd.to_datetime('2020-02-07', utc=True)


pipe = Pipeline(columns={
    'Close': USEquityPricing.close.latest,
},
screen = StaticAssets(symbols(['IBM', 'F', 'AAPL']))
)

stocks = spe.run_pipeline(pipe, pipe_start, pipe_end)
stocks

[2020-07-22 13:49:34.916852] INFO: sharadar_db_bundle: Compute values for pipeline from 2020-02-03 to 2020-02-07.


Close
2020-02-03 00:00:00+00:00 Equity(199059 [AAPL])  309.51
                          Equity(199623 [IBM])   143.73
                          Equity(199713 [F])       8.82
2020-02-04 00:00:00+00:00 Equity(199059 [AAPL])  308.66
                          Equity(199623 [IBM])   146.27
                          Equity(199713 [F])       8.98
2020-02-05 00:00:00+00:00 Equity(199059 [AAPL])  318.85
                          Equity(199623 [IBM])   149.11
                          Equity(199713 [F])       9.18
2020-02-06 00:00:00+00:00 Equity(199059 [AAPL])  321.45
                          Equity(199623 [IBM])   156.33
                          Equity(199713 [F])       8.31
2020-02-07 00:00:00+00:00 Equity(199059 [AAPL])  324.44
                          Equity(199623 [IBM])   153.52
                          Equity(199713 [F])       8.25

In [3]:
%%time
pipe_mkt_cap = Pipeline(columns={
    'mkt_cap': MarketCap()
},
)

start_time = time.time()
stocks = spe.run_pipeline(pipe_mkt_cap, pipe_start, pipe_end)
stocks.shape

[2020-07-03 20:55:18.348540] INFO: sharadar_db_bundle: Compute pipeline values in chunks of 120 days.
[2020-07-03 20:55:18.350274] INFO: sharadar_db_bundle: Compute values for pipeline from 2020-02-03 to 2020-02-07.
[2020-07-03 20:55:18.356553] INFO: sharadar_db_bundle: Computing term 1 of 5 [AssetExists()]
[2020-07-03 20:55:18.357422] INFO: sharadar_db_bundle: Term already in workspace: no computation needed
[2020-07-03 20:55:18.358058] INFO: sharadar_db_bundle: Computing term 2 of 5 [Fundamentals{'field': 'sharesbas_arq'}]
[2020-07-03 20:55:20.697451] INFO: sharadar_db_bundle: Elapsed time: 0:00:02.339389
[2020-07-03 20:55:20.698366] INFO: sharadar_db_bundle: Computing term 3 of 5 [Fundamentals{'field': 'sharefactor_arq'}]
[2020-07-03 20:55:21.922237] INFO: sharadar_db_bundle: Elapsed time: 0:00:01.223812
[2020-07-03 20:55:21.923614] INFO: sharadar_db_bundle: Computing term 4 of 5 [USEquityPricing.close::float64]
[2020-07-03 20:55:21.930245] INFO: sharadar_db_bundle: Elapsed time: 0:

(48891, 1)

In [4]:
%%time
pipe_mkt_cap_ev = Pipeline(columns={
    'mkt_cap': MarketCap(),
    'ev': EV(),
    'debt': Fundamentals(field='debtusd_arq'),
    'cash': Fundamentals(field='cashnequsd_arq')
},
screen = StaticAssets(symbols(['IBM', 'F', 'AAPL']))
)

stocks = spe.run_pipeline(pipe_mkt_cap_ev, pipe_start, pipe_end)
stocks

[2020-07-03 20:55:28.480497] INFO: sharadar_db_bundle: Compute pipeline values in chunks of 120 days.
[2020-07-03 20:55:28.482117] INFO: sharadar_db_bundle: Compute values for pipeline from 2020-02-03 to 2020-02-07.
[2020-07-03 20:55:28.489209] INFO: sharadar_db_bundle: Computing term 1 of 9 [AssetExists()]
[2020-07-03 20:55:28.490335] INFO: sharadar_db_bundle: Term already in workspace: no computation needed
[2020-07-03 20:55:28.491284] INFO: sharadar_db_bundle: Computing term 2 of 9 [Fundamentals{'field': 'sharesbas_arq'}]
[2020-07-03 20:55:28.496066] INFO: sharadar_db_bundle: Elapsed time: 0:00:00.004796
[2020-07-03 20:55:28.496953] INFO: sharadar_db_bundle: Computing term 3 of 9 [Fundamentals{'field': 'sharefactor_arq'}]
[2020-07-03 20:55:28.501872] INFO: sharadar_db_bundle: Elapsed time: 0:00:00.004916
[2020-07-03 20:55:28.502813] INFO: sharadar_db_bundle: Computing term 4 of 9 [Fundamentals{'field': 'cashnequsd_arq'}]
[2020-07-03 20:55:30.464896] INFO: sharadar_db_bundle: Elapsed

cash          debt  \
2020-02-03 00:00:00+00:00 Equity(199059 [AAPL])  3.977100e+10  1.082920e+11   
                          Equity(199623 [IBM])   1.022500e+10  7.149400e+10   
                          Equity(199713 [F])     2.052300e+10  1.541110e+11   
2020-02-04 00:00:00+00:00 Equity(199059 [AAPL])  3.977100e+10  1.082920e+11   
                          Equity(199623 [IBM])   1.022500e+10  7.149400e+10   
                          Equity(199713 [F])     2.052300e+10  1.541110e+11   
2020-02-05 00:00:00+00:00 Equity(199059 [AAPL])  3.977100e+10  1.082920e+11   
                          Equity(199623 [IBM])   1.022500e+10  7.149400e+10   
                          Equity(199713 [F])     2.052300e+10  1.541110e+11   
2020-02-06 00:00:00+00:00 Equity(199059 [AAPL])  3.977100e+10  1.082920e+11   
                          Equity(199623 [IBM])   1.022500e+10  7.149400e+10   
                          Equity(199713 [F])     1.750400e+10  1.553070e+11   
2020-02-07 00:00:00+00:00 Equity(199059 [AAPL])  3.977100e+10  1.082920e+11   
                          Equity(199623 [IBM])   1.022500e+10  7.149400e+10   
                          Equity(199713 [F])     1.750400e+10  1.553070e+11   

                                                           ev       mkt_cap  
2020-02-03 00:00:00+00:00 Equity(199059 [AAPL])  1.422776e+12  1.354255e+12  
                          Equity(199623 [IBM])   1.885617e+11  1.272927e+11  
                          Equity(199713 [F])     1.685584e+11  3.497039e+10  
2020-02-04 00:00:00+00:00 Equity(199059 [AAPL])  1.419057e+12  1.350536e+12  
                          Equity(199623 [IBM])   1.908112e+11  1.295422e+11  
                          Equity(199713 [F])     1.691928e+11  3.560478e+10  
2020-02-05 00:00:00+00:00 Equity(199059 [AAPL])  1.463643e+12  1.395122e+12  
                          Equity(199623 [IBM])   1.933264e+11  1.320574e+11  
                          Equity(199713 [F])     1.699858e+11  3.639775e+10  
2020-02-06 00:00:00+00:00 Equity(199059 [AAPL])  1.475019e+12  1.406498e+12  
                          Equity(199623 [IBM])   1.997207e+11  1.384517e+11  
                          Equity(199713 [F])     1.709586e+11  3.315556e+10  
2020-02-07 00:00:00+00:00 Equity(199059 [AAPL])  1.491471e+12  1.422950e+12  
                          Equity(199623 [IBM])   1.986668e+11  1.373978e+11  
                          Equity(199713 [F])     1.707192e+11  3.291617e+10

In [4]:
%%time
pipe = Pipeline(columns={
    'mkt_cap': MarketCap(),
    'ev': EV(),
    'debt': Fundamentals(field='debtusd_arq'),
    'cash': Fundamentals(field='cashnequsd_arq')
},
screen = StaticAssets(symbols(['IBM', 'F', 'AAPL']))
)

stocks = spe.run_pipeline(pipe, pipe_start, pipe_end)
stocks

[2020-07-22 13:49:47.051972] INFO: sharadar_db_bundle: Compute values for pipeline from 2020-02-03 to 2020-02-07.
CPU times: user 1.72 s, sys: 752 ms, total: 2.47 s
Wall time: 3.54 s


cash          debt  \
2020-02-03 00:00:00+00:00 Equity(199059 [AAPL])  3.977100e+10  1.082920e+11   
                          Equity(199623 [IBM])   1.022500e+10  7.149400e+10   
                          Equity(199713 [F])     2.052300e+10  1.541110e+11   
2020-02-04 00:00:00+00:00 Equity(199059 [AAPL])  3.977100e+10  1.082920e+11   
                          Equity(199623 [IBM])   1.022500e+10  7.149400e+10   
                          Equity(199713 [F])     2.052300e+10  1.541110e+11   
2020-02-05 00:00:00+00:00 Equity(199059 [AAPL])  3.977100e+10  1.082920e+11   
                          Equity(199623 [IBM])   1.022500e+10  7.149400e+10   
                          Equity(199713 [F])     2.052300e+10  1.541110e+11   
2020-02-06 00:00:00+00:00 Equity(199059 [AAPL])  3.977100e+10  1.082920e+11   
                          Equity(199623 [IBM])   1.022500e+10  7.149400e+10   
                          Equity(199713 [F])     1.750400e+10  1.553070e+11   
2020-02-07 00:00:00+00:00 Equity(199059 [AAPL])  3.977100e+10  1.082920e+11   
                          Equity(199623 [IBM])   1.022500e+10  7.149400e+10   
                          Equity(199713 [F])     1.750400e+10  1.553070e+11   

                                                           ev       mkt_cap  
2020-02-03 00:00:00+00:00 Equity(199059 [AAPL])  1.422776e+12  1.354255e+12  
                          Equity(199623 [IBM])   1.885617e+11  1.272927e+11  
                          Equity(199713 [F])     1.685584e+11  3.497039e+10  
2020-02-04 00:00:00+00:00 Equity(199059 [AAPL])  1.419057e+12  1.350536e+12  
                          Equity(199623 [IBM])   1.908112e+11  1.295422e+11  
                          Equity(199713 [F])     1.691928e+11  3.560478e+10  
2020-02-05 00:00:00+00:00 Equity(199059 [AAPL])  1.463643e+12  1.395122e+12  
                          Equity(199623 [IBM])   1.933264e+11  1.320574e+11  
                          Equity(199713 [F])     1.699858e+11  3.639775e+10  
2020-02-06 00:00:00+00:00 Equity(199059 [AAPL])  1.475019e+12  1.406498e+12  
                          Equity(199623 [IBM])   1.997207e+11  1.384517e+11  
                          Equity(199713 [F])     1.709586e+11  3.315556e+10  
2020-02-07 00:00:00+00:00 Equity(199059 [AAPL])  1.488102e+12  1.419581e+12  
                          Equity(199623 [IBM])   1.972321e+11  1.359631e+11  
                          Equity(199713 [F])     1.707192e+11  3.291617e+10

In [5]:
stocks = spe.run_pipeline(pipe, pipe_start)
stocks

[2020-07-22 13:50:05.540844] INFO: sharadar_db_bundle: Compute values for pipeline from 2020-02-03 to 2020-02-03.


cash          debt  \
2020-02-03 00:00:00+00:00 Equity(199059 [AAPL])  3.977100e+10  1.082920e+11   
                          Equity(199623 [IBM])   1.022500e+10  7.149400e+10   
                          Equity(199713 [F])     2.052300e+10  1.541110e+11   

                                                           ev       mkt_cap  
2020-02-03 00:00:00+00:00 Equity(199059 [AAPL])  1.422776e+12  1.354255e+12  
                          Equity(199623 [IBM])   1.885617e+11  1.272927e+11  
                          Equity(199713 [F])     1.685584e+11  3.497039e+10